# Implementation of WGAN-GP
Reference: https://arxiv.org/pdf/1704.00028.pdf


In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd

In [2]:
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
def to_cuda(x):
    x = Variable(x)
    if torch.cuda.is_available():
        return x.cuda()
    return x

In [4]:
def to_onehot(x, num_classes=10):
    assert isinstance(x, int) or isinstance(x.data, (torch.cuda.LongTensor, torch.LongTensor))    
    if isinstance(x, int):
        c = to_cuda(torch.zeros(1, num_classes))
        c.data[0][x] = 1
    else:
        c = to_cuda(torch.FloatTensor(x.size(0), num_classes))
        c.zero_()
        c.scatter_(1, x, 1) # dim, index, src value
    return c

In [5]:
def n_critic(step, nc=5):
    if step < 25 or step % 500 == 0:
        return 100
    return nc

In [6]:
class Discriminator(nn.Module):
    """
        Convolutional Discriminator for MNIST
    """
    def __init__(self, input_size=784, num_classes=1):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        y_ = x.view(x.size(0), -1)
        y_ = self.layer1(y_)
        y_ = self.layer2(y_)
        y_ = self.layer3(y_)
        return y_

In [7]:
class Generator(nn.Module):
    """
        Convolutional Generator for MNIST
    """
    def __init__(self, input_size=100, num_classes=784):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
            nn.Tanh()
        )
        
    def forward(self, x):
        y_ = self.layer(x)
        y_ = y_.view(x.size(0), 1, 28, 28)
        return y_

In [8]:
D = Discriminator().cuda()
G = Generator().cuda()

In [9]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))]
)

In [10]:
mnist = datasets.MNIST(root='../data/', train=True, transform=transform, download=True)

In [11]:
batch_size = 64

In [12]:
data_loader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True, drop_last=True)

In [13]:
D_opt = torch.optim.Adam(D.parameters(), lr=0.0001, betas=(0.0, 0.9))
G_opt = torch.optim.Adam(G.parameters(), lr=0.0001, betas=(0.0, 0.9))

In [50]:
max_epoch = 500 # need more than 500 epochs for training
step = 0
g_step = 0
n_noise = 100
p_coeff = 10 # lambda

In [15]:
D_labels = to_cuda(torch.ones(batch_size, 1)) # Discriminator Label to real
D_fakes = to_cuda(torch.zeros(batch_size, 1)) # Discriminator Label to fake

In [ ]:
for epoch in range(max_epoch):
    for idx, (images, labels) in enumerate(data_loader):
        step += 1
                   
        ## Training Discriminator
        # Real data
        x = to_cuda(images)
        
        # Sampling
        z = to_cuda(torch.randn(batch_size, n_noise))
        x_fake = G(z)
                
        # Gradient Penalty (e.g. gradients w.r.t x_penalty)
        eps = to_cuda(torch.rand(batch_size, 1, 1, 1)) # x shape: (64, 1, 28, 28)
        x_penalty = eps*x + (1-eps)*x_fake
        x_penalty = x_penalty.view(x_penalty.size(0), -1)
        p_outputs = D(x_penalty)
        xp_grad = autograd.grad(outputs=p_outputs, inputs=x_penalty, grad_outputs=D_labels,
                                create_graph=True, retain_graph=True, only_inputs=True)
        grad_penalty = p_coeff * torch.mean(torch.pow(torch.norm(xp_grad[0], 2, 1) - 1, 2))
        
        # Wasserstein loss
        x_outputs = D(x)
        z_outputs = D(x_fake)
        D_x_loss = torch.mean(x_outputs)
        D_z_loss = torch.mean(z_outputs)
        D_loss = D_z_loss - D_x_loss + grad_penalty
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()
                    
        if step % n_critic(g_step) == 0:
            g_step += 1
            # Training Generator
            z = to_cuda(torch.randn(batch_size, n_noise))
            z_outputs = D(G(z))
            G_loss = torch.mean(-z_outputs, 0)

            D.zero_grad()
            G.zero_grad()
            G_loss.backward()
            G_opt.step()
        
        if step % 1000 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, -D_loss.data[0], -G_loss.data[0]))

In [98]:
import scipy.misc

In [160]:
# generation to image
z = to_cuda(torch.randn(1, n_noise))
fakeimg = G(z).view(28, 28)
img = fakeimg.cpu().data.numpy()
scipy.misc.toimage(img)

/home/yangyangii/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """


In [15]:
def save_checkpoint(state, file_name='checkpoint.pth.tar'):
    torch.save(state, file_name)

In [16]:
# Saving params.
# torch.save(D.state_dict(), 'D_c.pkl')
# torch.save(G.state_dict(), 'G_c.pkl')
save_checkpoint({'epoch': epoch + 1, 'state_dict':D.state_dict(), 'optimizer' : D_opt.state_dict()}, 'D_gp.pth.tar')
save_checkpoint({'epoch': epoch + 1, 'state_dict':G.state_dict(), 'optimizer' : G_opt.state_dict()}, 'G_gp.pth.tar')